In [20]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import serial
import pandas as pd
import os
import tqdm
import time

# Helper functions

In [16]:
def get_vr(XR, YR, ZR):
    XVR = -0.04838958374914194 * XR + -0.0293806035280404 * YR + 0.012915074248156884 * ZR + 0.002196403571507596
    YVR = 0.003961772264919089 * XR + -0.011191365720206181 * YR + -0.023897067525894065 * ZR + -0.0037991112598711654
    ZVR = 0.013618067240529615 * XR + 0.20871153042749133 * YR + 0.17603269331043395 * ZR + -0.03927379520069467

    return XVR, YVR, ZVR

def get_dist(rssi):
    dist = -1.6623285827510927 * rssi + 278.37820524137487
    return dist

In [17]:
# helper function
# Create a function to calculate the Euclidean distance between rows
def calculate_distance(df_baseline, row):
    return np.sqrt(
        (df_baseline['roll1'] - row['roll1']) ** 2 +
        (df_baseline['pitch1'] - row['pitch1']) ** 2 +
        (df_baseline['heading1'] - row['heading1']) ** 2 +
        (df_baseline['roll2'] - row['roll2']) ** 2 +
        (df_baseline['pitch2'] - row['pitch2']) ** 2 +
        (df_baseline['heading2'] - row['heading2']) ** 2
    )

# Optimized function to find the closest row in df_baseline for each row in df_vr and subtract values
def find_and_subtract(df_vr, df_baseline):
    # Iterate over rows in df_vr
    for index, row in df_vr.iterrows():
        # Calculate the distance between the current row and all rows in df_baseline
        distances = calculate_distance(df_baseline, row)
        
        # Find the index of the closest row in df_baseline
        closest_index = distances.idxmin()
        
        # Perform the subtraction on mx1, my1, mz1, mx2, my2, mz2
        df_vr.at[index, 'mx1'] -= df_baseline.at[closest_index, 'mx1']
        df_vr.at[index, 'my1'] -= df_baseline.at[closest_index, 'my1']
        df_vr.at[index, 'mz1'] -= df_baseline.at[closest_index, 'mz1']
        df_vr.at[index, 'mx2'] -= df_baseline.at[closest_index, 'mx2']
        df_vr.at[index, 'my2'] -= df_baseline.at[closest_index, 'my2']
        df_vr.at[index, 'mz2'] -= df_baseline.at[closest_index, 'mz2']

    return df_vr

df_baseline = pd.read_csv('../calibration/data/data_baseline.csv')

# TENS Settings

In [10]:
# inverse mapping
def get_tens(dist):
    tMode = 2
    int_min = 0
    int_max = 9

    dist_min = 0
    dist_max = 150 # in mm

    # Ensure the distance is within the bounds
    dist = max(min(dist, dist_max), dist_min)
    
    # Inverse linear mapping: as distance decreases, intensity increases
    tIntensity = int_min + (int_max - int_min) * (dist_max - dist) / (dist_max - dist_min)
    
    return int(tMode), int(tIntensity)


# VIBRO Settings

In [40]:
# inverse mapping
def get_vibro(dist):

    vibIntMin = 0
    vibIntMax = 255

    pulseDuration = 500 # in ms
    pulseSpacing = 500    # in ms

    max_freq = 10 # in Hz
    min_freq = 250 # in Hz


    dist_min = 0
    dist_max = 150 # in mm

    # Ensure the distance is within the bounds
    dist = max(min(dist, dist_max), dist_min)
    
    # Inverse linear mapping: as distance decreases, intensity increases
    tIntensity = vibIntMin + (vibIntMax - vibIntMin) * (dist_max - dist) / (dist_max - dist_min)
    # tIntensity = 255

    # # inverse map the frequency
    # tFreq = min_freq + (max_freq - min_freq) * (dist_max - dist) / (dist_max - dist_min)

    # # get the pulse duration and spacing
    # tPeriod = 1000/tFreq
    # pulseDuration = tPeriod/2
    # pulseSpacing = tPeriod/2
    
    
    return int(tIntensity), int(pulseDuration), int(pulseSpacing)


# Create connections

In [30]:
# Serial object (macOS)
PORT = '/dev/cu.usbserial-21440'
BAUD = 115200
try:
    GATEWAY = serial.Serial(PORT, BAUD)
    print('GateWay opened')
except serial.SerialException:
    print('GateWay not available')

GateWay opened


# Get imu data

In [44]:
while True:
    distance_df = pd.DataFrame(columns=['gx1', 'gy1', 'gz1', 'ax1', 'ay1', 'az1', 'mx1', 'my1', 'mz1',
                                        'roll1', 'pitch1', 'heading1', 
                                        'gx2', 'gy2', 'gz2', 'ax2', 'ay2', 'az2', 'mx2', 'my2', 'mz2',
                                        'roll2', 'pitch2', 'heading2'])
    for i in range(10):

        # Send 's' to sensor 1 to start streaming data
        GATEWAY.write(b'1')
        
        # Wait for data from SENSOR1
        while GATEWAY.in_waiting < 1:
            pass
        data1 = GATEWAY.readline().decode('utf-8').strip()
        
        # Send 's' to sensor 2 to start streaming data
        GATEWAY.write(b'2')

        # Wait for data from SENSOR2
        while GATEWAY.in_waiting < 1:
            pass
        data2 = GATEWAY.readline().decode('utf-8').strip()
        
        #print(data1)
        #print(data2)

        # Split and convert SENSOR1 data to float
        data1 = [float(i) for i in data1.split(',')]
        
        # Split and convert SENSOR2 data to float
        data2 = [float(i) for i in data2.split(',')]
        
        
        # Combine both sensor data and append distance
        combined_data = data1 + data2

        # Create a temporary dataframe for the current sample
        temp_df = pd.DataFrame([combined_data], columns=['gx1', 'gy1', 'gz1', 'ax1', 'ay1', 'az1', 'mx1', 'my1', 'mz1', 
                                                        'roll1', 'pitch1', 'heading1', 
                                                        'gx2', 'gy2', 'gz2', 'ax2', 'ay2', 'az2', 'mx2', 'my2', 'mz2', 
                                                        'roll2', 'pitch2', 'heading2'])
        
        # Append the temporary dataframe to the distance dataframe
        distance_df = pd.concat([distance_df, temp_df], ignore_index=True)
    
    # # Subtract baseline values from the current sample
    # temp_df = find_and_subtract(temp_df, df_baseline)

    # # Get the VR values
    # XVR, YVR, ZVR = get_vr(temp_df['mx2'], temp_df['my2'], temp_df['mz2'])

    # # Get the distance
    # Xx = temp_df['mx1'] - XVR
    # Yy = temp_df['my1'] - YVR
    # Zz = temp_df['mz1'] - ZVR

    # # RSSI
    # rssi = np.sqrt(Xx**2 + Yy**2 + Zz**2)

    # average each column of df
    df_avg = distance_df.mean()

    #df_avg = find_and_subtract(df_avg, df_baseline)

    mx2, my2, mz2 = get_vr(df_avg['mx2'], df_avg['my2'], df_avg['mz2'])

    # subtract mx2, my2, mz2 from mx1, my1, mz1
    Mx = df_avg['mx1'] - mx2
    My = df_avg['my1'] - my2
    Mz = df_avg['mz1'] - mz2

    # Calculate RSSI
    rssi = np.sqrt(Mx ** 2 + My ** 2 + Mz ** 2)

    # Distance
    dist = get_dist(rssi)

    print("Estimated distance: ", dist)

    

    # # Get the intensity
    # targetMode, targetIntensity = get_tens(dist)
    # # vibro
    vibIntensity, pulseDuration, pulseSpacing = get_vibro(dist)

    # # Send the intensity to the tens unit with format 'EtargetMode,targetIntensity'
    # GATEWAY.write(f'E{targetMode},{targetIntensity}'.encode())
    # # Send the intensity to the vibro unit with format 'VvibIntensity,pulseDuration,pulseSpacing'
    GATEWAY.write(f'V0,{vibIntensity},{pulseDuration},{pulseSpacing}\n'.encode())

    # print(f'E{targetMode},{targetIntensity}')

    #time.sleep(0.05)
    

Estimated distance:  8.975282839668125
Estimated distance:  8.694747429289123
Estimated distance:  9.222741741156824
Estimated distance:  8.548634890515359
Estimated distance:  36.68312599835477
Estimated distance:  8.635937340574287
Estimated distance:  64.06682557816876
Estimated distance:  8.62684790178065
Estimated distance:  8.280202461212014
Estimated distance:  8.126747142128579
Estimated distance:  8.366670689783575
Estimated distance:  8.399394848292104
Estimated distance:  8.71655538684638
Estimated distance:  8.820995434699057
Estimated distance:  8.943754937588778
Estimated distance:  64.26345944029794
Estimated distance:  9.213390704563551
Estimated distance:  9.206699130024958
Estimated distance:  9.717580523610422
Estimated distance:  9.845024607795551
Estimated distance:  10.031313041009696
Estimated distance:  9.177932818514478
Estimated distance:  9.226169779993313
Estimated distance:  9.215922321736855
Estimated distance:  8.749783790140896
Estimated distance:  9.052

KeyboardInterrupt: 

In [26]:
# close the serial connection
GATEWAY.close()

In [31]:
# test code for tens
GATEWAY.write(f'V0,255,500,500\n'.encode())

15